In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('spotify_millsongdata.csv')

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.isnull().sum()
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [5]:
df['text'][0]

"Down the side streets and the avenue  \r\nThere be sisters walkin' two by two  \r\nTheir dresses and their shoes are new  \r\nBut their hearts are weary through and through  \r\nAnd it's a long way into the light of the day  \r\nWhile the jukebox and the radio play  \r\n  \r\nWhere the days turn into the nights  \r\nPeople move into the sounds and sights  \r\nLike the moth is drawn into the lights  \r\nLike the tight-rope walker into the heights  \r\nIt's in their hearts, it's in their hips,  \r\nIt's in their feet, it's on their lips  \r\n  \r\nTonight's the night I'm gonna make you mine  \r\nGonna dance right out onto the edge of time  \r\nWhen the sound starts pumpin'  \r\nAnd the lights are flashin'  \r\nAnd my hearts-a-thumpin'  \r\nAnd I feel the passion  \r\nAnd the world is right there waiting  \r\nAt my finger tips  \r\nDisco... apocalypse  \r\n  \r\nIn the dawn the city seems to sigh  \r\nAnd the hungry hear their children cry  \r\nPeople watch the time go by  \r\nThey do th

In [6]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [29]:
import re

def regex_tokenization(text):
    # Split by non-word characters
    tokens = re.findall(r'\b\w+\b', text.lower())
    return " ".join(tokens)

# Apply on your dataframe
df['text'] = df['text'].apply(regex_tokenization)


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [23]:
similarity[0]

array([1.        , 0.04006198, 0.03068936, ..., 0.05563225, 0.03671798,
       0.07532001])

In [33]:
print(df.head(10))

           artist                                song  \
0  Jackson Browne                    Disco Apocalypse   
1   Elvis Presley                           By And By   
2            UB40   Things Ain't Like They Used To Be   
3     Dean Martin        By The Time I Get To Phoenix   
4        Flo-Rida                               Never   
5      Gary Numan                            Miracles   
6         Ramones  Do You Remember Rock N Roll Radio?   
7          Israel                 Jesus At The Center   
8         Ramones                          My Sharona   
9    Loretta Lynn            I Don't Wanna Play House   

                                                text  
0  down the side streets and the avenue there be ...  
1  well children by and by when the morning comes...  
2  chorus are you wondering why no one talks any ...  
3  by the time i get to phoenix she ll be risin s...  
4  never give up never hold you head down never f...  
5  i don t believe in miracles i don t bel

In [34]:
df[df['song'] == 'By The Time I Get To Phoenix'].index[0]

3

In [35]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [36]:
recommendation('By The Time I Get To Phoenix')

['By The Time I Get To Phoenix',
 "I'll Be There",
 'Give Me Time',
 '60 Miles An Hour',
 "I'll Be There",
 'Jackson 5 Medley',
 "I'll Be Back For More",
 'Promises',
 "I'm Your Man",
 "I'm Your Man",
 "I'll Get By (As Long As I Have You)",
 'Elsewhere',
 "I'll Fight For You",
 "You'll Think Of Me",
 'In Your Pocket',
 "Lover's Will",
 'Maybe Tomorrow',
 'Loved By You',
 "I'll Have To Say I Love You In A Song",
 'One Day']

In [37]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))

In [38]:
pickle.load(open('df.pkl', 'rb'))

,artist,song,text
0,Jackson Browne,Disco Apocalypse,down the side streets and the avenue there be ...
1,Elvis Presley,By And By,well children by and by when the morning comes...
2,UB40,Things Ain't Like They Used To Be,chorus are you wondering why no one talks any ...
3,Dean Martin,By The Time I Get To Phoenix,by the time i get to phoenix she ll be risin s...
4,Flo-Rida,Never,never give up never hold you head down never f...
...,...,...,...
4995,Reba Mcentire,I'll Still Be Loving You,changing my life with your love has been so ea...
4996,INXS,The Indian Song,and i was lost for words in your arms attempti...
4997,Erasure,Don't Say No,collecting conversation from the silence of th...
4998,Olivia Newton-John,Behind That Locked Door,harrison why are you still crying your pain is...
